# GWAS: Regenie for single variant and burden analysis in a public data-set

Here we show a how to run a genome wide association analysis (GWAS) using regenie with our `regenie.ipynb` pipeline using a public dataset from 1000 genomes. 

This tutorial has been created to be run on a single computer (a desktop, laptop, or a single interactive node on a cluster). The pipeline has been implemented in SoS workflow language and can be configured to run in parallel on a high performance computing cluster environment. Please read the [SoS documentation](https://vatlab.github.io/sos-docs/doc/user_guide/host_setup.html) on how to configure the software and workflow to efficiently perform the analysis for real-world data.

## Introduction

Over the past few years, there have been many tools created to run GWAS with increasing number of samples and genetic variants. Aditionally, these softwares use linear mixed models (LMM) to account for population structure, sample relatedness and estimate the genetic architecture of complex traits. BoltLMM, FastGWA and [Regenie](https://rgcgithub.github.io/regenie/) are some of them. 

Regenie can perform GWAS's in very large datases such as the UK Biobank.

## Regenie properties

1. Works with quantitative and binary traits (case-control), including those with unbalanced samples case-control ratio. Supports Firth logistic regression and SPA for binary traits.
2. Uses linear mixed models (LMM) for quantitative traits and generalized LMM (GLMM) for binary traits, allowing for the inclusion of related individuals. 
3. Offers the advantage of processing multiple phenotypes at the same time. However you should be aware that quantitave or binary traits need to be analyzed separately. Also, the proportion of missing data should be similar for all of the traits analyzed at once, since it can impact predictions. Authors of regenie recommend to analyze traits in groups that have similar missingness patterns with resonably low amount of missingness (<15%).
4. It can handle different types of data (microarray, imputed, exome sequencing) and formats (PLINK:bed/bim/fam, PLINK2:pgen/pvar/psam and BGEN).
5. It is desgined to handle a large number of samples ~500K. Regenie is not appropiate to analyze small samples
6. Integrates covariates into the model.

## Method

Regenie works in two different steps that are independent from each other. 

### Step 1. 

The first step is where it fits the null regression model, for this regenie uses a subset of genetic markers that capture a good proportion of the phenotype variance that is attributable to genetic effects

- Level 0: Ridge regression applied to block of SNPs to reduce dimensions
- Level 1: Linear or logistic ridge regressions within cross validation scheme

### Step 2.

A larger set of genetic markers is used to test for association, using a linear or logistic regression, with the traits conditional upon the prediction from the regression model on step 1. This is done by using the Leave One Chromosome Out (LOCO) scheme to avoid proximal contamination. 

## Pre-requisites

In order to run this workflow you will need to have installed [docker](https://docs.docker.com/get-docker/) in you local computer previously, and have created an account.


## Data 

## Genotype data

For this exercise we will use data from 1000 genomes pubicly available database downloaded from [MAGMA](https://ctg.cncr.nl/software/magma).The number of individuals in this data is n=489.

Please note that this is only an example dataset for real life applications a much larger sample size is necessary. Also due to the use of different populations the use of principal components may be necessary to avoid increased type I error. 

```
1000G.EUR.mwe.pruned.fam
1000G.EUR.mwe.pruned.bim
1000G.EUR.mwe.pruned.bed
```

The genotype file has been pruned in advance to make sure the excercise can be finished in a reasonable amount of time. For reproducibility these are the commands that have been used for pruning

```
plink2 --bfile 1000G.EUR --indep-pairwise 100 10 0.01 --out 1000G.EUR.mwe
plink2 --bfile 1000G.EUR --extract 1000G.EUR.mwe.prune.in --make-bed --out 1000G.EUR.mwe.pruned
```


## Phenotype file

For the 1000G data there is not phenotype data available. Therefore we have create a phenotype `x` by randomly assigning study subjects as cases and controls. Therefore the data is generated under the null of no association. 

```
library(dplyr)
EUR_subset_fam <- read.table('1000G.EUR.fam',  head=F)
colnames(EUR_subset_fam) <- c("FID", "IID", "mother", "father", "sex", "phenotype")
EUR_subset_fam$x <- sample(c(0,1), replace=TRUE, size=489)
EUR_subset_fam$sex <- sample(c(0,1), replace=TRUE, size=489)
pheno <- EUR_subset_fam %>%
    select('FID', "IID", "sex", "x")
head(pheno)
write.table(pheno,'1000G.EUR.pheno.x', sep="\t",row.names=F, quote=F)
```

In [ ]:
tree

## 1. Runing the single variant association analysis

**1. Prepare the data**

In this example, we are going to use a subset of genetic markers from the data to fit the null model of step 1. 

This process has been already integrated in the [LMM pipeline](https://github.com/cumc/bioworkflows/blob/master/GWAS/LMM.ipynb) under the `regenie_qc` step. It is recommended to use the data from the genotype array in this step, as it provides better estimates of the phenotype variance. 

Note: make sure you don't have any SNVs with a very low minor allele count (MAC) for the single variant analysis, otherwise regenie will give an error message. 

Let the parameters for the initial quality control be:

- maf_filter=0.01, we will keep only variants with MAF >1%
- geno_filter=0.1, we will remove variants with > 10% genotypes missing
- mind_filter=0.1, we will remove individals with > 10% genotypes missing 
- hwe_filter=5e-08, we will remove variants exceding a p-value for HWE > 5E-08

The output produced in this part consist of two files:

`1000G.EUR.mwe.pruned.qc_pass.id` and `1000G.EUR.mwe.pruned.snplist`, these contain the list of individuals and variants to keep in the step 1 of regenie. 

In this MWE we keep n=102,497 variants and n=489 samples that pass our QC filters. 


**2. Fitting the null**

In this step, we will use the genotype file and we will tell regenie which samples and variants to keep based on the results of our QC. 

- bfile=1000G.EUR.mwe.pruned, we will use a subset of the genotyped genetic markers to calculate the predictions.
- block_size=1000 this will tell the program in how many "chunks" to divide the genotype file to make predictions

The output produced in this step corresponds to: 

- A set of files (depending on the number of phenotypes analyzed) containing genomic predictions for each phenotype from Step 1 `1000G.EUR.pheno_x.regenie_1.loco`
- A file called `1000G.EUR.pheno_x.regenie_pred.list` listing the locations of the prediction files

**3. Association analysis**

In this step, namely step 2 of regenie, it will perform the single variant association analysis with each of the phenotypes. 

Note: a convenient quality of REGENIE is that step 1 and 2 are decoupled meaning that you can use all of the traits used in step 1 or just a subset of them for the association analysis. Also, you can test the association using array, exomed or imputed variants. 

For this example, we will use our phenotype x with the following parameters:

- trait='bt' Here you define whether your trait is binary of quantitative. 
- covarCol=sex. In our particular case we will only use sex as covariate. However, if you have more than one categorical covariate you can input them here separeted by comma
- qCovarCol We will leave this variable empty in our analysis, but as above you can give a list of quantitative variables. 
- minMac=5 this flag is used to tell the program which minimum minor allele count (MAC) to use when testing variants, default value is 5. In real data applications you may want to set threshold depending on the power to detect associations based on your sample size. 

Depending on the type of data you are using, there are other useful parameters that you can explore. For example, with imputed data you may want to set the minimum info score to use (`--minINFO`), with quantitative phenotypes you may choose to use an inverse rank normalization (`--apply-rint`), for binary traits you can decide whether to use Firth (`--firth`) or Saddle Point Approximation (`--spa`) corrections.

The output produced in this step corresponds to:

- Summary statistics in a file with `*.regenie` extension, if option `-gz` was used this files will be compressed `*.regenie.gz`

### REGENIE: single variant association analysis

We set the work directory to mwe_regenie folder (to be created by the workflow).

In [ ]:
wd='mwe_regenie'

Now, we run the workflow `regenie` with the specific parameters. Please note that in this particular case, we are using the same file for calculating the predictions on Step 1 of regenie and for the association itself. However, for real life applications you may use different sets of files. For example genotype array data in bfile and exome sequencing data in the genoFile. 

Note: when you execute the cell using (shift + enter), a star will appear on the left side inside the box. When the star dissapears it means that the run has finished. 


In [ ]:
sos run regenie.ipynb regenie \
    --cwd $wd/output \
    --bfile $wd/1000G.EUR.mwe.pruned.bed \
    --maf-filter 0.01 \
    --geno-filter 0.1 \
    --mind-filter 0.1 \
    --hwe_filter 5e-08 \
    --genoFile $wd/1000G.EUR.mwe.pruned.bed\
    --phenoFile $wd/1000G.EUR.pheno.x \
    --formatFile $wd/regenie_template.yml\
    --phenoCol x\
    --covarCol sex \
    --qCovarCol \
    --numThreads 8 \
    --bsize 1000 \
    --trait bt \
    --minMAC 5 \
    --reverse_log_p \
    --p-filter 1 

Now let's visualize the results of the association

#### Manhattan plot

In [ ]:
%preview -s png mwe_regenie/output/1000G.EUR.pheno_x.regenie.manhattan.png

#### Q-Q plot

In [ ]:
%preview -s png mwe_regenie/output/1000G.EUR.pheno_x.regenie.qq.png

**Question:** What is the inflation (lambda GC) value for this run? 


#### Lambda GC value

You can get a general idea of the number or cases/controls, the number of variants analyzed and the lambda GC values by taking a look at the markdown file generated in the `output` folder. 

In [ ]:
cat $wd/output/1000G.EUR.pheno_x.regenie.analysis_summary.md

## 2. Regenie : burden test

Regenie offers the functionality of performing rare-variant aggregate association analysis in the form of burden tests.  

You can combine rare-variants in a gene or a region, using functional annotations to create masks that are then tested for association (as in a single variant analysis) with a phenotype. 

### Input files:

1. You will need to provide an `annotation file` that is formatted: variant_id, gene/region, functional annotation (e.g. LoF, missense). You can use VEP or ANNOVAR to generate this information and then format it accordingly. 
2. Provide the `set-list-file`: this file contains a list of variants within each gene/region that's used when building the masks. The format is: gene/region name, chromosome, start_position, list of variants in the gene/region separated by comma. 
3. Optional: provide a file with genes/regions that you want to include or exclude from your analysis.
4. Optional: provide an `allele-frequency` file to use when creating the masks. By default the allele frequency is computed from the sample. In our case we are providing an allele-frequency file, obtained from gnomAD exome frequencies. We will use the AF_nfe field which contains the allele frequencies from exome data for non Finnish Europeans available in gnomAD. 
5. Mask file: this is a text file that contains the name of the mask and the type of annotations to use when building it (one mask per line). E.g. 

```
mask1 LoF,missense
```

6. You need to provide the `--aaf-bins` cut-off in the parameters. This refers to the AAF upper bounds to use when creating the masks. By default regenie_burden produces results for singletons and if you set `--aaf-bins` to be for example 0.01 it will create masks from  [0,0.01] and singletons. 

Additionally, you can choose the way the mask are built among these options:
- using the maximum number of ALT alleles across sites ('max'; the default)
- using the sum of ALT alleles ('sum')
- using a maximum threshold of 2 ('comphet')

![Rules to build mask](https://rgcgithub.github.io/regenie/img/mask_rules.png)


For this MWE we have already generated the annotation, the set list, the allele frequency and the mask files that are provided in the `data` folder. 

### REGENIE: burden test

In [ ]:
sos run regenie.ipynb regenie_burden \
    --cwd $wd/output_burden \
    --bfile $wd/1000G.EUR.mwe.pruned.bed \
    --maf-filter 0.01 \
    --geno-filter 0.1 \
    --mind-filter 0.1 \
    --hwe_filter 5e-08 \
    --genoFile $wd/1000G.EUR.mwe.pruned.bed \
    --phenoFile  $wd/1000G.EUR.pheno.x \
    --formatFile $wd/regenie_template.yml\
    --phenoCol x \
    --covarCol sex \
    --qCovarCol \
    --numThreads 8 \
    --bsize 1000 \
    --trait bt \
    --minMAC 1 \
    --reverse_log_p \
    --p-filter 1 \
    --anno_file $wd/*.anno_file\
    --aaf_file $wd/*.aff_file\
    --set_list $wd/*.set_list_file\
    --mask_file $wd/*.mask_file\
    --build_mask max\
    --aaf_bins  0.01 -s build

### Output files

After running regenie burden you will find the following output files in the `output_burden` folder:

* `1000G.EUR.pheno_x.regenie_burden.mask1.0.01.snp_stats.gz` This file contains the results only for mask1 and 0.01 bin
* `1000G.EUR.pheno_x.regenie_burden.mask1.0.01.manhattan.png` Manhattan pot
* `1000G.EUR.pheno_x.regenie_burden.mask1.0.01.qq.png` Q-Q plot
* `1000G.EUR.pheno_x.regenie_burden.mask1.0.01.analysis_summary.md` Markdown file with summarized information

You can see that manhattan and qqplots have been generated for mask1 with the 0.01 bin combination. 

### Manhattan plot

In [ ]:
%preview -s png mwe_regenie/output_burden/1000G.EUR.pheno_x.regenie_burden.mask1.0.01.manhattan.png

### Q-Q plot

In [ ]:
%preview -s png mwe_regenie/output_burden/1000G.EUR.pheno_x.regenie_burden.mask1.0.01.qq.png